In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# first load libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os , math , re , random
import glob 

import pydicom
import cv2
import time
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import albumentations

In [ ]:
!pip install -q efficientnet
import tensorflow as tf
import tensorflow.keras.layers as L

from tensorflow.keras.callbacks import EarlyStopping,LearningRateScheduler, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.models import Model,Sequential


import efficientnet.tfkeras as efn
from tensorflow.keras.applications import DenseNet121, DenseNet201
from tensorflow.keras.applications import vgg16
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications import MobileNet , MobileNetV2
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras import optimizers

!pip install tensorflow-addons=='0.9.1'
import tensorflow_addons as tfa

In [ ]:
tf.__version__

In [ ]:
from kaggle_datasets import KaggleDatasets

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import classification_report
from sklearn.utils import shuffle


In [ ]:
# !git clone https://github.com/chentinghao/download_google_drive.git
    
# !python ./download_google_drive/download_gdrive.py 1HoHknBnHFhdKkQkeOkEyrRz_Ete47WMY best_model_b0_vloss=0.0162_vauc=0.9325.h5
# !python ./download_google_drive/download_gdrive.py 1dNqwBxPVVDc2KQVtCunuRWYqYTrR48Hg best_model_b3_vacc=0.9159_vloss=0.0182_vauc=0.9332.h5
# !python ./download_google_drive/download_gdrive.py 1h8T4M9gdxJLSbED2yMfUUFHdtC3-TwXL best_model_b5_vacc=0.9239_vloss=0.0165_vauc=0.9347.h5

OVERVIEW THE DATA

In [ ]:
# for reproducible results :
def seed_everything(seed=13):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['TF_KERAS'] = '1'
    random.seed(seed)
    
seed_everything(42)

In [ ]:
try :
    tpu=tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on :',tpu.master())
except ValueError :
    tpu = None

if tpu :    
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else :
    strategy = tf.distribute.get_strategy()
    
print('Replicas :',strategy.num_replicas_in_sync)  

In [ ]:
AUTO  = tf.data.experimental.AUTOTUNE

GCS_DS_PATH2 = KaggleDatasets().get_gcs_path('siim-isic-melanoma-classification')
GCS_DS_PATH1 = KaggleDatasets().get_gcs_path('512x512-melanoma-tfrecords-70k-images')
EPOCHS = 10
BATCH_SIZE = 8 * strategy.num_replicas_in_sync 
img_size = 468 # 468 for effnet b5
SEED =  42
nb_classes = 1

In [ ]:
train = pd.read_csv('../input/siim-isic-melanoma-classification/train.csv')
test = pd.read_csv('../input/siim-isic-melanoma-classification/test.csv')
sub = pd.read_csv('../input/siim-isic-melanoma-classification/sample_submission.csv')

train_paths = train.image_name.apply(lambda x : GCS_DS_PATH1 + '/train/' +x + '.jpg').values
test_paths = test.image_name.apply(lambda x : GCS_DS_PATH1 + '/test/' +x + '.jpg').values

In [ ]:
test.head()

In [ ]:
train.head()

In [ ]:
train.isnull().sum()

In [ ]:
train['age_approx'].fillna(train['age_approx'].mean(),inplace = True)
train['sex'].fillna('unknown_sex',inplace = True)
train['anatom_site_general_challenge'].fillna('unknown_anatom',inplace = True)


In [ ]:
print(train['sex'].value_counts())
print('*'*100)
print(train['diagnosis'].value_counts())
print('*'*100)
print(train['anatom_site_general_challenge'].value_counts())
print(train['benign_malignant'].value_counts())
print(train['target'].value_counts())

In [ ]:
one_hot_sex = pd.get_dummies(train.sex, prefix='sex')
one_hot_diagnosis = pd.get_dummies(train.diagnosis , prefix = 'disgnosis')
one_hot_anatom = pd.get_dummies(train.anatom_site_general_challenge ,prefix = 'anatom')

train = train.join(one_hot_sex)
train = train.join(one_hot_diagnosis)
train = train.join(one_hot_anatom)

train = train.drop(['sex','diagnosis','anatom_site_general_challenge','benign_malignant'],axis = 1)
train['id'] = train['patient_id'].map(lambda x : int(x[3:]))
train.drop('patient_id',axis = 1,inplace = True)


one_hot_sex = pd.get_dummies(test.sex, prefix='sex')
one_hot_anatom = pd.get_dummies(test.anatom_site_general_challenge ,prefix = 'anatom')

test = test.join(one_hot_sex)
test = test.join(one_hot_diagnosis)
test = test.join(one_hot_anatom)

test = test.drop(['sex','anatom_site_general_challenge'],axis = 1)
test['id'] = test['patient_id'].map(lambda x : int(x[3:]))
test.drop('patient_id',axis = 1,inplace = True)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
import cv2
dup_patients_test = test[test.id.duplicated() == True]
unique_patient_test_ids = list(set(dup_patients_test['id']))
patient = test[test['id'] == unique_patient_test_ids[2]] 
patient = patient.reset_index(drop=True)
images = []
for i in range(len(patient)) :
    img = '../input/siim-isic-melanoma-classification/jpeg/test/'+patient['image_name'][i]+'.jpg'
    n = cv2.imread(img)
    n = cv2.cvtColor(n, cv2.COLOR_BGR2GRAY)
    images.append(n)

import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
for i in range(len(images)) :
    plt.subplot(8,8,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(images[i])

In [ ]:
bool_random_brightness = False # 0.902 no improvement
bool_random_contrast =False  # doesn't improve
bool_random_hue = False
bool_random_saturation = False

gridmask_rate = 0.4 #improve 
cutmix_rate = 0.4 #improve
mixup_rate = 0 #doesn't improve
rotation = False
random_blackout = False
crop_size = 0
transforms = False

In [ ]:
# batch
def cutmix_b0(image, label, PROBABILITY = cutmix_rate, img_size=224):
    # input image - is a batch of images of size [n,dim,dim,3] not a single image of [dim,dim,3]
    # output - a batch of images with cutmix applied
    
    DIM = img_size    
    imgs = []; labs = []
    
    for j in range(BATCH_SIZE):
        
        #random_uniform( shape, minval=0, maxval=None)        
        # DO CUTMIX WITH PROBABILITY DEFINED ABOVE
        P = tf.cast(tf.random.uniform([], 0, 1) <= PROBABILITY, tf.int32)
        
        # CHOOSE RANDOM IMAGE TO CUTMIX WITH
        k = tf.cast(tf.random.uniform([], 0, BATCH_SIZE), tf.int32)
        
        # CHOOSE RANDOM LOCATION
        x = tf.cast(tf.random.uniform([], 0, DIM), tf.int32)
        y = tf.cast(tf.random.uniform([], 0, DIM), tf.int32)
        
        # Beta(1, 1)
        b = tf.random.uniform([], 0, 1) # this is beta dist with alpha=1.0
        

        WIDTH = tf.cast(DIM * tf.math.sqrt(1-b),tf.int32) * P
        ya = tf.math.maximum(0,y-WIDTH//2)
        yb = tf.math.minimum(DIM,y+WIDTH//2)
        xa = tf.math.maximum(0,x-WIDTH//2)
        xb = tf.math.minimum(DIM,x+WIDTH//2)
        
        # MAKE CUTMIX IMAGE
        one = image[j,ya:yb,0:xa,:]
        two = image[k,ya:yb,xa:xb,:]
        three = image[j,ya:yb,xb:DIM,:]        
        #ya:yb
        middle = tf.concat([one,two,three],axis=1)

        img = tf.concat([image[j,0:ya,:,:],middle,image[j,yb:DIM,:,:]],axis=0)
        imgs.append(img)
        
        # MAKE CUTMIX LABEL
        a = tf.cast(WIDTH*WIDTH/DIM/DIM,tf.float32)
        lab1 = label[j,]
        lab2 = label[k,]
        labs.append((1-a)*lab1 + a*lab2)

    image2 = tf.reshape(tf.stack(imgs),(BATCH_SIZE,DIM,DIM,3))
    label2 = tf.reshape(tf.stack(labs),(BATCH_SIZE, nb_classes))
    return image2, label2


# batch
def cutmix_b3(image, label, PROBABILITY = cutmix_rate, img_size=320):
    # input image - is a batch of images of size [n,dim,dim,3] not a single image of [dim,dim,3]
    # output - a batch of images with cutmix applied
    
    DIM = img_size    
    imgs = []; labs = []
    
    for j in range(BATCH_SIZE):
        
        #random_uniform( shape, minval=0, maxval=None)        
        # DO CUTMIX WITH PROBABILITY DEFINED ABOVE
        P = tf.cast(tf.random.uniform([], 0, 1) <= PROBABILITY, tf.int32)
        
        # CHOOSE RANDOM IMAGE TO CUTMIX WITH
        k = tf.cast(tf.random.uniform([], 0, BATCH_SIZE), tf.int32)
        
        # CHOOSE RANDOM LOCATION
        x = tf.cast(tf.random.uniform([], 0, DIM), tf.int32)
        y = tf.cast(tf.random.uniform([], 0, DIM), tf.int32)
        
        # Beta(1, 1)
        b = tf.random.uniform([], 0, 1) # this is beta dist with alpha=1.0
        

        WIDTH = tf.cast(DIM * tf.math.sqrt(1-b),tf.int32) * P
        ya = tf.math.maximum(0,y-WIDTH//2)
        yb = tf.math.minimum(DIM,y+WIDTH//2)
        xa = tf.math.maximum(0,x-WIDTH//2)
        xb = tf.math.minimum(DIM,x+WIDTH//2)
        
        # MAKE CUTMIX IMAGE
        one = image[j,ya:yb,0:xa,:]
        two = image[k,ya:yb,xa:xb,:]
        three = image[j,ya:yb,xb:DIM,:]        
        #ya:yb
        middle = tf.concat([one,two,three],axis=1)

        img = tf.concat([image[j,0:ya,:,:],middle,image[j,yb:DIM,:,:]],axis=0)
        imgs.append(img)
        
        # MAKE CUTMIX LABEL
        a = tf.cast(WIDTH*WIDTH/DIM/DIM,tf.float32)
        lab1 = label[j,]
        lab2 = label[k,]
        labs.append((1-a)*lab1 + a*lab2)

    image2 = tf.reshape(tf.stack(imgs),(BATCH_SIZE,DIM,DIM,3))
    label2 = tf.reshape(tf.stack(labs),(BATCH_SIZE, nb_classes))
    return image2, label2


# batch
def cutmix_b5(image, label, PROBABILITY = cutmix_rate, img_size=468):
    # input image - is a batch of images of size [n,dim,dim,3] not a single image of [dim,dim,3]
    # output - a batch of images with cutmix applied
    
    DIM = img_size    
    imgs = []; labs = []
    
    for j in range(BATCH_SIZE):
        
        #random_uniform( shape, minval=0, maxval=None)        
        # DO CUTMIX WITH PROBABILITY DEFINED ABOVE
        P = tf.cast(tf.random.uniform([], 0, 1) <= PROBABILITY, tf.int32)
        
        # CHOOSE RANDOM IMAGE TO CUTMIX WITH
        k = tf.cast(tf.random.uniform([], 0, BATCH_SIZE), tf.int32)
        
        # CHOOSE RANDOM LOCATION
        x = tf.cast(tf.random.uniform([], 0, DIM), tf.int32)
        y = tf.cast(tf.random.uniform([], 0, DIM), tf.int32)
        
        # Beta(1, 1)
        b = tf.random.uniform([], 0, 1) # this is beta dist with alpha=1.0
        

        WIDTH = tf.cast(DIM * tf.math.sqrt(1-b),tf.int32) * P
        ya = tf.math.maximum(0,y-WIDTH//2)
        yb = tf.math.minimum(DIM,y+WIDTH//2)
        xa = tf.math.maximum(0,x-WIDTH//2)
        xb = tf.math.minimum(DIM,x+WIDTH//2)
        
        # MAKE CUTMIX IMAGE
        one = image[j,ya:yb,0:xa,:]
        two = image[k,ya:yb,xa:xb,:]
        three = image[j,ya:yb,xb:DIM,:]        
        #ya:yb
        middle = tf.concat([one,two,three],axis=1)

        img = tf.concat([image[j,0:ya,:,:],middle,image[j,yb:DIM,:,:]],axis=0)
        imgs.append(img)
        
        # MAKE CUTMIX LABEL
        a = tf.cast(WIDTH*WIDTH/DIM/DIM,tf.float32)
        lab1 = label[j,]
        lab2 = label[k,]
        labs.append((1-a)*lab1 + a*lab2)

    image2 = tf.reshape(tf.stack(imgs),(BATCH_SIZE,DIM,DIM,3))
    label2 = tf.reshape(tf.stack(labs),(BATCH_SIZE, nb_classes))
    return image2, label2

In [ ]:
def mixup(image, label, PROBABILITY = mixup_rate, img_size=224):
    # input image - is a batch of images of size [n,dim,dim,3] not a single image of [dim,dim,3]
    # output - a batch of images with mixup applied
    DIM = img_size
    
    imgs = []; labs = []
    for j in range(BATCH_SIZE):
        
        # CHOOSE RANDOM
        k = tf.cast( tf.random.uniform([],0,BATCH_SIZE),tf.int32)
        a = tf.random.uniform([],0,1) # this is beta dist with alpha=1.0

        #mixup
        P = tf.cast(tf.random.uniform([], 0, 1) <= PROBABILITY, tf.int32)
        if P==1:
            a=0.
        
        # MAKE MIXUP IMAGE
        img1 = image[j,]
        img2 = image[k,]
        imgs.append((1-a)*img1 + a*img2)
        
        # MAKE CUTMIX LABEL
        lab1 = label[j,]
        lab2 = label[k,]
        labs.append((1-a)*lab1 + a*lab2)
            
    # RESHAPE HACK SO TPU COMPILER KNOWS SHAPE OF OUTPUT TENSOR (maybe use Python typing instead?)
    image2 = tf.reshape(tf.stack(imgs),(BATCH_SIZE,DIM,DIM,3))
    label2 = tf.reshape(tf.stack(labs),(BATCH_SIZE,nb_classes))
    return image2,label2

In [ ]:
def transform(image, inv_mat, image_shape):
    h, w, c = image_shape
    cx, cy = w//2, h//2
    new_xs = tf.repeat( tf.range(-cx, cx, 1), h)
    new_ys = tf.tile( tf.range(-cy, cy, 1), [w])
    new_zs = tf.ones([h*w], dtype=tf.int32)
    old_coords = tf.matmul(inv_mat, tf.cast(tf.stack([new_xs, new_ys, new_zs]), tf.float32))
    old_coords_x, old_coords_y = tf.round(old_coords[0, :] + w//2), tf.round(old_coords[1, :] + h//2)
    clip_mask_x = tf.logical_or(old_coords_x<0, old_coords_x>w-1)
    clip_mask_y = tf.logical_or(old_coords_y<0, old_coords_y>h-1)
    clip_mask = tf.logical_or(clip_mask_x, clip_mask_y)
    old_coords_x = tf.boolean_mask(old_coords_x, tf.logical_not(clip_mask))
    old_coords_y = tf.boolean_mask(old_coords_y, tf.logical_not(clip_mask))
    new_coords_x = tf.boolean_mask(new_xs+cx, tf.logical_not(clip_mask))
    new_coords_y = tf.boolean_mask(new_ys+cy, tf.logical_not(clip_mask))
    old_coords = tf.cast(tf.stack([old_coords_y, old_coords_x]), tf.int32)
    new_coords = tf.cast(tf.stack([new_coords_y, new_coords_x]), tf.int64)
    rotated_image_values = tf.gather_nd(image, tf.transpose(old_coords))
    rotated_image_channel = list()
    for i in range(c):
        vals = rotated_image_values[:,i]
        sparse_channel = tf.SparseTensor(tf.transpose(new_coords), vals, [h, w])
        rotated_image_channel.append(tf.sparse.to_dense(sparse_channel, default_value=0, validate_indices=False))
    return tf.transpose(tf.stack(rotated_image_channel), [1,2,0])

def GridMask(image_height, image_width, d1, d2, rotate_angle=1, ratio=0.5):
    h, w = image_height, image_width
    hh = int(np.ceil(np.sqrt(h*h+w*w)))
    hh = hh+1 if hh%2==1 else hh
    d = tf.random.uniform(shape=[], minval=d1, maxval=d2, dtype=tf.int32)
    l = tf.cast(tf.cast(d,tf.float32)*ratio+0.5, tf.int32)

    st_h = tf.random.uniform(shape=[], minval=0, maxval=d, dtype=tf.int32)
    st_w = tf.random.uniform(shape=[], minval=0, maxval=d, dtype=tf.int32)

    y_ranges = tf.range(-1 * d + st_h, -1 * d + st_h + l)
    x_ranges = tf.range(-1 * d + st_w, -1 * d + st_w + l)

    for i in range(0, hh//d+1):
        s1 = i * d + st_h
        s2 = i * d + st_w
        y_ranges = tf.concat([y_ranges, tf.range(s1,s1+l)], axis=0)
        x_ranges = tf.concat([x_ranges, tf.range(s2,s2+l)], axis=0)

    x_clip_mask = tf.logical_or(x_ranges < 0 , x_ranges > hh-1)
    y_clip_mask = tf.logical_or(y_ranges < 0 , y_ranges > hh-1)
    clip_mask = tf.logical_or(x_clip_mask, y_clip_mask)

    x_ranges = tf.boolean_mask(x_ranges, tf.logical_not(clip_mask))
    y_ranges = tf.boolean_mask(y_ranges, tf.logical_not(clip_mask))

    hh_ranges = tf.tile(tf.range(0,hh), [tf.cast(tf.reduce_sum(tf.ones_like(x_ranges)), tf.int32)])
    x_ranges = tf.repeat(x_ranges, hh)
    y_ranges = tf.repeat(y_ranges, hh)

    y_hh_indices = tf.transpose(tf.stack([y_ranges, hh_ranges]))
    x_hh_indices = tf.transpose(tf.stack([hh_ranges, x_ranges]))

    y_mask_sparse = tf.SparseTensor(tf.cast(y_hh_indices, tf.int64),  tf.zeros_like(y_ranges), [hh, hh])
    y_mask = tf.sparse.to_dense(y_mask_sparse, 1, False)

    x_mask_sparse = tf.SparseTensor(tf.cast(x_hh_indices, tf.int64), tf.zeros_like(x_ranges), [hh, hh])
    x_mask = tf.sparse.to_dense(x_mask_sparse, 1, False)

    mask = tf.expand_dims( tf.clip_by_value(x_mask + y_mask, 0, 1), axis=-1)

    mask = random_rotate(mask, rotate_angle, [hh, hh, 1])
    mask = tf.image.crop_to_bounding_box(mask, (hh-h)//2, (hh-w)//2, image_height, image_width)

    return mask

def apply_grid_mask(image, image_shape, PROBABILITY = gridmask_rate):
    AugParams = {
        'd1' : 100,
        'd2': 160,
        'rotate' : 45,
        'ratio' : 0.3
    }
    
        
    mask = GridMask(image_shape[0], image_shape[1], AugParams['d1'], AugParams['d2'], AugParams['rotate'], AugParams['ratio'])
    if image_shape[-1] == 3:
        mask = tf.concat([mask, mask, mask], axis=-1)
        mask = tf.cast(mask,tf.float32)
        P = tf.cast(tf.random.uniform([], 0, 1) <= PROBABILITY, tf.int32)
    if P==1:
        return image*mask
    else:
        return image

def gridmask_b0(img_batch, label_batch, img_size=224):
    return apply_grid_mask(img_batch, (img_size, img_size, 3)), label_batch

def gridmask_b3(img_batch, label_batch, img_size=320):
    return apply_grid_mask(img_batch, (img_size, img_size, 3)), label_batch

def gridmask_b5(img_batch, label_batch, img_size=468):
    return apply_grid_mask(img_batch, (img_size, img_size, 3)), label_batch

In [ ]:
def random_rotate(image, angle, image_shape):
    def get_rotation_mat_inv(angle):
        # transform to radian
        angle = math.pi * angle / 180
        cos_val = tf.math.cos(angle)
        sin_val = tf.math.sin(angle)
        one = tf.constant([1], tf.float32)
        zero = tf.constant([0], tf.float32)
        rot_mat_inv = tf.concat([cos_val, sin_val, zero, -sin_val, cos_val, zero, zero, zero, one], axis=0)
        rot_mat_inv = tf.reshape(rot_mat_inv, [3,3])
        return rot_mat_inv
    angle = float(angle) * tf.random.normal([1],dtype='float32')
    rot_mat_inv = get_rotation_mat_inv(angle)
    return transform(image, rot_mat_inv, image_shape)

In [ ]:
def decode_image(image_data, img_size=468):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range
    image = tf.reshape(image, [*IMAGE_SIZE, 3]) # explicit size needed for TPU
    image = tf.image.resize(image, [img_size,img_size])
    return image


def data_augment(image, label=None, seed = 2020):
    # data augmentation. Thanks to the dataset.prefetch(AUTO) statement in the next function (below),
    # this happens essentially for free on TPU. Data pipeline code is executed on the "CPU" part
    # of the TPU while the TPU itself is computing gradients.
    image = tf.image.random_flip_left_right(image,seed=seed)
    image = tf.image.random_flip_up_down(image,seed=seed)
    image = tf.image.random_brightness(image,0.2)
    image = tf.image.random_contrast(image,0.6,1.4)
    image = tf.image.random_hue(image,0.07)
    image = tf.image.random_saturation(image,0.5,1.5)
        
    if label == None :
        return image
    else :
        return image, label 

In [ ]:
#TRAINING_FILENAMES = tf.io.gfile.glob(GCS_DS_PATH2 + '/tfrecords/train*')
#TEST_FILENAMES = tf.io.gfile.glob(GCS_DS_PATH2 + '/tfrecords/test*')

TRAINING_FILENAMES = tf.io.gfile.glob(GCS_DS_PATH1 + '/train*')
#TEST_FILENAMES = tf.io.gfile.glob('../input/siim-isic-melanoma-classification/tfrecords/test*')
# VAL_FILENAMES = TRAINING_FILENAMES[int(0.9*len(TRAINING_FILENAMES)):]
# TRAINING_FILENAMES = TRAINING_FILENAMES[:int(0.9*len(TRAINING_FILENAMES))]
TEST_FILENAMES = tf.io.gfile.glob(GCS_DS_PATH1 + '/test*')

#TRAINING_FILENAMES = tf.io.gfile.glob('../input/512x512-melanoma-tfrecords-70k-images/train*')
#TEST_FILENAMES = tf.io.gfile.glob('../input/512x512-melanoma-tfrecords-70k-images/test*')

IMAGE_SIZE = [512, 512] 

In [ ]:
#Read train tf Records :
def read_labeled_tfrecord(example, img_size=468):
    
    
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "image_name": tf.io.FixedLenFeature([], tf.string),
        "target": tf.io.FixedLenFeature([], tf.int64),  # shape [] means single element
    }
 
    
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'], img_size)
    idnum = example['image_name']
    label = tf.cast(example['target'], tf.float32)
    return image, label

#Read test tf Records :
def read_unlabeled_tfrecord(example, img_size=468):
    UNLABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "image_name": tf.io.FixedLenFeature([], tf.string),  # shape [] means single element
        # class is missing, this competitions's challenge is to predict flower classes for the test dataset
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'], img_size)
    idnum = example['image_name']
    return image, idnum # returns a dataset of image(s)

def load_dataset(filenames, labeled=True, ordered=False, img_size=468):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # disregarding data order. Order does not matter since we will be shuffling the data anyway.

    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed

    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(lambda x: read_labeled_tfrecord(x, img_size) if labeled else read_unlabeled_tfrecord(x, img_size), num_parallel_calls=AUTO)
    # returns a dataset of (image, label) pairs if labeled=True or (image, id) pairs if labeled=False
    return dataset

   
    
def get_training_dataset(img_size=468, name='b0'):
    dataset = load_dataset(TRAINING_FILENAMES, labeled=True, img_size=img_size)
    dataset = dataset.map(data_augment, num_parallel_calls=AUTO)
    dataset = dataset.repeat() # the training dataset must repeat for several epochs
    dataset = dataset.shuffle(2048, seed=img_size)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)# prefetch next batch while training (autotune prefetch buffer size)
#     if name == 'b0':
#         dataset = dataset.map(cutmix_b0,   num_parallel_calls = AUTO) 
#         dataset = dataset.map(gridmask_b0, num_parallel_calls = AUTO) 
#     if name == 'b3':
#         dataset = dataset.map(cutmix_b3,   num_parallel_calls = AUTO) 
#         dataset = dataset.map(gridmask_b3, num_parallel_calls = AUTO) 
#     if name == 'b5':
#         dataset = dataset.map(cutmix_b5,   num_parallel_calls = AUTO) 
#         dataset = dataset.map(gridmask_b5, num_parallel_calls = AUTO) 
    return dataset   

def get_val_dataset(img_size=468):
    dataset = load_dataset(VAL_FILENAMES, labeled=True, img_size=img_size)
    dataset = dataset.repeat() # the training dataset must repeat for several epochs
    dataset = dataset.shuffle(2048, seed=img_size)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
        
    return dataset

def get_test_dataset(ordered=False,img_size=468,aug=False):
    dataset = load_dataset(TEST_FILENAMES, labeled=False, img_size=img_size, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
        
    return dataset

In [ ]:
def count_data_items(filenames):
    # the number of data items is written in the name of the .tfrec files, i.e. flowers00-230.tfrec = 230 data items
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

NUM_TRAINING_IMAGES = count_data_items(TRAINING_FILENAMES)
NUM_TEST_IMAGES = count_data_items(TEST_FILENAMES)
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE
print('Dataset: {} training images, {} unlabeled test images'.format(NUM_TRAINING_IMAGES, NUM_TEST_IMAGES))

In [ ]:
reduce_lr =  ReduceLROnPlateau(monitor = "val_loss", factor = 0.5, patience = 3,
  verbose = 0, mode = "auto", epsilon = 1e-04, cooldown = 0,
  min_lr = 1e-6)

In [ ]:
es = EarlyStopping(monitor = "val_los" , verbose = 1 , mode = 'min' , patience = 10 )

In [ ]:
mc_b5 = ModelCheckpoint('best_model_b5.h5', monitor = 'loss' , mode = 'min', verbose = 1 , save_best_only = True)
mc_b3 = ModelCheckpoint('best_model_b3.h5', monitor = 'loss' , mode = 'min', verbose = 1 , save_best_only = True)
mc_b0 = ModelCheckpoint('best_model_b0.h5', monitor = 'loss' , mode = 'min', verbose = 1 , save_best_only = True)

In [ ]:
focal_loss = True
label_smoothing = 0
SWA = False

In [ ]:
def get_model_generalized(name,trainable_layers=25,opt='adam',lr=0.001,img_size=468, auc=None):
    if name == 'EfficientNetB7' :
        base_model = efn.EfficientNetB7(weights='imagenet',
                                        include_top = False,
                                        input_shape=(img_size,img_size,3)
                                       )
    elif name == 'EfficientNetB5' :
        base_model = efn.EfficientNetB5(weights='imagenet',
                                        include_top = False,
                                        input_shape=(img_size,img_size,3)
                                       )
    elif name == 'EfficientNetB3' :
        base_model = efn.EfficientNetB3(weights='imagenet',
                                        include_top = False,
                                        input_shape=(img_size,img_size,3)
                                       )
    elif name == 'EfficientNetB0' :
        base_model = efn.EfficientNetB0(weights='imagenet',
                                        include_top = False,
                                        input_shape=(img_size,img_size,3)
                                       )    

    base_model.trainable = True
    for layer in base_model.layers[:-trainable_layers] :
        layer.trainable = True
    layer = base_model.output
    layer = L.GlobalAveragePooling2D()(layer)
    layer = L.Dense(img_size*2,activation=tf.keras.activations.softplus)(layer)
    layer = L.Dropout(0.5)(layer)
    layer = L.BatchNormalization()(layer)
    predictions = L.Dense(nb_classes,activation='sigmoid')(layer)
   # predictions = tf.cast(predictions,tf.float32)
    model = Model(inputs = base_model.input, outputs=predictions)
    
    if focal_loss : 
        loss= tfa.losses.SigmoidFocalCrossEntropy(reduction=tf.keras.losses.Reduction.AUTO)
    elif label_smoothing :
        loss = tf.keras.losses.BinaryCrossentropy(label_smoothing=label_smoothing)
    else :
        loss = 'binary_crossentropy'
    
    
    if opt == 'RMSprop' :
        opt = optimizers.RMSprop(learning_rate = lr)
    elif SWA == True :
        opt = tf.keras.optimizers.Adam(lr=1e-4) # roll back
        opt = tfa.optimizers.SWA(opt)
    else :
        opt =  tf.keras.optimizers.Adam(lr=1e-4) # roll back

    model.compile(optimizer=opt,loss=loss,metrics=['accuracy', auc])  
    return model

In [ ]:
with strategy.scope() :
    auc = tf.keras.metrics.AUC(name='auc')
    model_b0 = get_model_generalized('EfficientNetB0', img_size=224, auc=auc)

history = model_b0.fit(
    get_training_dataset(img_size=224, name='b0'),
    steps_per_epoch=STEPS_PER_EPOCH,
#     validation_data=get_val_dataset(img_size=224),
#     validation_steps=STEPS_PER_EPOCH // 10,
    epochs=20,
    callbacks = [reduce_lr, es, mc_b0],
    #class_weight = classweights,
)

In [ ]:
with strategy.scope() :
    auc = tf.keras.metrics.AUC(name='auc')
    model_b3 = get_model_generalized('EfficientNetB3', img_size=320, auc=auc)

history = model_b3.fit(
    get_training_dataset(img_size=320, name='b3'),
    steps_per_epoch=STEPS_PER_EPOCH,
#     validation_data=get_val_dataset(img_size=320),
#     validation_steps=STEPS_PER_EPOCH // 10,
    epochs=20,
    callbacks = [reduce_lr, es, mc_b3],
    #class_weight = classweights,
)

In [ ]:
with strategy.scope() :
    auc = tf.keras.metrics.AUC(name='auc')
    model_b5 = get_model_generalized('EfficientNetB5', img_size=468, auc=auc)

history = model_b5.fit(
    get_training_dataset(img_size=468, name='b5'),
    steps_per_epoch=STEPS_PER_EPOCH,
#     validation_data=get_val_dataset(img_size=468),
#     validation_steps=STEPS_PER_EPOCH // 10,
    epochs=20,
    callbacks = [reduce_lr, es, mc_b5],
    #class_weight = classweights,
)

In [ ]:
# !git clone https://github.com/chentinghao/download_google_drive.git
    
# !python ./download_google_drive/download_gdrive.py 1HoHknBnHFhdKkQkeOkEyrRz_Ete47WMY best_model_b0_vloss=0.0162_vauc=0.9325.h5
# !python ./download_google_drive/download_gdrive.py 1dNqwBxPVVDc2KQVtCunuRWYqYTrR48Hg best_model_b3_vacc=0.9159_vloss=0.0182_vauc=0.9332.h5
# !python ./download_google_drive/download_gdrive.py 1h8T4M9gdxJLSbED2yMfUUFHdtC3-TwXL best_model_b5_vacc=0.9239_vloss=0.0165_vauc=0.9347.h5
# !python ./download_google_drive/download_gdrive.py 1h8T4M9gdxJLSbED2yMfUUFHdtC3-TwXL best_model_b5_vacc=0.9239_vloss=0.0165_vauc=0.9347.h5

In [ ]:
test_ds_b5 = get_test_dataset(ordered=True, img_size=468)
test_ds_b3 = get_test_dataset(ordered=True, img_size=320)
test_ds_b0 = get_test_dataset(ordered=True, img_size=224)

# models_name = ['best_model_b0_vloss=0.0162_vauc=0.9325.h5', 
#                'best_model_b3_vacc=0.9159_vloss=0.0182_vauc=0.9332.h5', 
#                'best_model_b5_vacc=0.9239_vloss=0.0165_vauc=0.9347.h5']

models_name = ['best_model_b0.h5', 
               'best_model_b3.h5', 
               'best_model_b5.h5']

for name in models_name:
    if name[12] == '5':
        model_b5.load_weights(name)
    if name[12] == '3':
        model_b3.load_weights(name)
    if name[12] == '0':
        model_b0.load_weights(name)

print('Computing predictions...')
test_images_ds_b5 = test_ds_b5.map(lambda image, idnum: image)
test_images_ds_b3 = test_ds_b3.map(lambda image, idnum: image)
test_images_ds_b0 = test_ds_b0.map(lambda image, idnum: image)

probabilities_b5 = model_b5.predict(test_images_ds_b5)
probabilities_b3 = model_b3.predict(test_images_ds_b3)
probabilities_b0 = model_b0.predict(test_images_ds_b0)

probabilities = (probabilities_b0 + probabilities_b3 + probabilities_b5) / 3

if label_smoothing :
    probabilities = LabelSmoothing(probabilities,label_smoothing)
    
probabilities_b5 =probabilities_b5.flatten()
probabilities_b3 =probabilities_b3.flatten()
probabilities_b0 =probabilities_b0.flatten()
probabilities =probabilities.flatten()

probabilities_list = [probabilities_b0, probabilities_b3, probabilities_b5]

print('Generating submission.csv file...')
test_ids_ds = test_ds_b0.map(lambda image, idnum: idnum).unbatch()
test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U')

pred_df_b5 = pd.DataFrame({'image_name': test_ids, 'target': probabilities_b5})
pred_df_b3 = pd.DataFrame({'image_name': test_ids, 'target': probabilities_b3})
pred_df_b0 = pd.DataFrame({'image_name': test_ids, 'target': probabilities_b0})
pred_df_mean = pd.DataFrame({'image_name': test_ids, 'target': probabilities})

for name in models_name:
    if name[12] == '5':
        pred_df_b5.to_csv('pred_df_b5_submission.csv', index=False)
    if name[12] == '3':
        pred_df_b3.to_csv('pred_df_b3_submission.csv', index=False)
    if name[12] == '0':
        pred_df_b0.to_csv('pred_df_b0_submission.csv', index=False)



In [ ]:
pred_df_mean.to_csv('pred_df_mean_submission_gamma-0.50.csv', index=False)
pred_df_mean.head()

In [ ]:
pred_df_mean.head()

In [ ]:
pred_df_mean[pred_df_mean['target'] > 0.5]

In [ ]:
import gc
del model_b5
del model_b3
del model_b0
gc.collect()

In [ ]:
# # GAMMA SEARCH
# for gamma in range(40, 61, 3):
#     gamma = gamma / 100
    
#     probabilities = probabilities - 0.5 + gamma
#     pred_df_gamma = pd.DataFrame({'image_name': test_ids, 'target': probabilities})
# #     sub_gamma = sub.merge(pred_df_gamma, on='image_name')
#     pred_df_gamma.to_csv('pred_df_mean_submission_gamma-{}.csv'.format(round(gamma, 2)), index=False)

In [ ]:
pred_df_b5_0936 = pd.read_csv('../input/melanoma-dif-sub/pl_0.936.csv')

# lets rank each prediction and then divide it by its max value to we have our predictions between 0 and 1
def rank_data(sub):
    sub['target'] = sub['target'].rank() / sub['target'].rank().max()
    return sub

pred_df_b5 = rank_data(pred_df_b5)
pred_df_b0 = rank_data(pred_df_b0)
pred_df_b3 = rank_data(pred_df_b3)
pred_df_b5_0936 = rank_data(pred_df_b5_0936)
pred_df_mean = rank_data(pred_df_mean)

pred_df_b5_0936.columns = ['image_name', 'target1']
pred_df_b5.columns = ['image_name', 'target2']
pred_df_mean.columns = ['image_name', 'target3']
pred_df_b3.columns = ['image_name', 'target4']
pred_df_b0.columns = ['image_name', 'target5']

f_sub = pred_df_mean.merge(pred_df_b5_0936, on = 'image_name').merge(pred_df_b3, on = 'image_name').merge(pred_df_b0, on = 'image_name').merge(pred_df_b5, on = 'image_name')
f_sub['target'] = f_sub['target1'] * 0.3 + f_sub['target2'] * 0.3 + f_sub['target3'] * 0.05 + f_sub['target4'] * 0.3 + f_sub['target5'] * 0.05
f_sub = f_sub[['image_name', 'target']]
f_sub.to_csv('blend_sub.csv', index = False)